# Day 11

In [72]:
test_data = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
"""


In [73]:
from adventofcode import AdventOfCode
import re
import pprint
from copy import deepcopy
import math
from tqdm import tqdm

class Day11(AdventOfCode):
    worry = False
    rounds = 20

    def get_input_values(self):
        lines = self.input_data.strip().split("\n")
        print(len(lines))
        monkeys={}
        for i in range(len(lines)):
            if "Monkey" in lines[i]:
                idx = len(monkeys)
                monkeys[idx] = {
                    "items": [int(x) for x in re.findall(r'(\d+)', lines[i+1])],
                    "operation": re.search(r'.*=\s+(.*)', lines[i+2]).groups()[-1],
                    "divisor": int(re.findall(r'(\d+)', lines[i+3])[-1]),
                    "target_true": int(re.findall(r'(\d+)', lines[i+4])[-1]),
                    "target_false": int(re.findall(r'(\d+)', lines[i+5])[-1]),
                    "activity": 0
                }
        return monkeys
            

    def solve(self):
        monkeys = self.get_input_values()
        # pprint.pprint(monkeys)

        for round in tqdm(range(self.rounds)):
            # print("Round ", round)
            for i in range(len(monkeys.keys())):
                # print(f"\tMonkey {i}:")
                m = monkeys[i]
                items_init = deepcopy(m["items"])
                new_items = []
                for j in range(len(items_init)):
                    m["activity"] += 1
                    old = items_init[j]
                    # print(f"\t\tMonkey inspects an item with a worry level of {old}.")
                    new = eval(m["operation"])
                    if not self.worry:
                        # print(f"\t\t\tWorry level is {m['operation']} to {new}.")
                        new //= 3
                    # print(f"\t\t\tMonkey gets bored with item. Worry level is divided by 3 to {new}.")
                    if (new % m["divisor"] == 0):
                        # print(f"\t\t\tCurrent worry level IS divisible by {m['divisor']}.")
                        monkeys[m["target_true"]]["items"] += [new]
                        # print(f"\t\t\tItem with worry level {new} is thrown to monkey {m['target_true']}..")
                    else:
                        # print(f"\t\t\tCurrent worry level is not divisible by {m['divisor']}.")
                        monkeys[m["target_false"]]["items"] += [new]
                        # print(f"\t\t\tItem with worry level {new} is thrown to monkey {m['target_false']}..")
                    m["items"].pop(m["items"].index(old))
            # pprint.pprint(monkeys)

        # for m in monkeys:
        #     print(f"Monkey {m} inspected items {monkeys[m]['activity']} times")
        two_most_actives = sorted([monkeys[m]['activity'] for m in monkeys], reverse=True)[:2]
        # print(f"Two most actives: {two_most_actives}")
        return math.prod(two_most_actives)





In [74]:
assert Day11(test_data).run() == 10605

27


100%|██████████| 20/20 [00:00<00:00, 5012.01it/s]

Running AdventOfCode : 10605


Part 1

In [75]:
from adventofcode import AdventOfCodeFromFileInput


challenge = AdventOfCodeFromFileInput(Day11, "input_11.txt")
challenge.run()

55


100%|██████████| 20/20 [00:00<00:00, 1176.54it/s]

Running AdventOfCode Day11: 50830


50830

Part 2

In [76]:
class Day11Part2(Day11):
    worry = True
    rounds = 10000

In [77]:
Day11Part2(test_data).run()
# 2713310158

27


  8%|▊         | 833/10000 [01:08<44:10,  3.46it/s]  